In [1]:
import sys
import numpy as np

from ac_segmentation.neurotorch.datasets.dataset import open_ZarrTensor

# zarr://http://bigkahuna.corp.alleninstitute.org/ACdata/Users/kevin/exaspim_ome_zarr/svc_axonal/H17_PO11_S9_20250221/exaSPIM.zarr/Pos0

In [11]:
## data set up
subfolder = 'H17_PO11_S8_20250226_exaSPIM_Pos0_cutout1'

## check whether the subfolder exists
import os
if not os.path.exists(f'/ACdata/Users/wanqing/data/image/{subfolder}'):
    os.makedirs(f'/ACdata/Users/wanqing/data/image/{subfolder}')

roi_bbox_NG = {
    "y": (18719, 20782),
    "x": (1688, 1905), 
    "z": (29031, 33102)
}

# Individual tile bounding boxes
tile_bboxes_NG = [
    {"z": (0, 69008), "y": (12062, 26254), "x": (-1, 5319)}
]

tile_paths = [
    "/ACdata/Users/kevin/exaspim_ome_zarr/svc_axonal/H17_PO11_S8_20250226/exaSPIM.zarr/Pos0/0/"
]

zarr_output_path = f'/ACdata/Users/wanqing/data/image/{subfolder}/{subfolder}.zarr'

tiff_output_path = f'/ACdata/Users/wanqing/data/image/{subfolder}/{subfolder}.tiff'

## save above metadata to a json file
import json
metadata = {
    "roi_bbox_NG": roi_bbox_NG,
    "tile_bboxes_NG": tile_bboxes_NG,
    "tile_paths": tile_paths,
    "zarr_output_path": zarr_output_path,
    "tiff_output_path": tiff_output_path
}

with open(f'/ACdata/Users/wanqing/data/image/{subfolder}/{subfolder}_metadata.json', 'w') as f:
    json.dump(metadata, f)

In [12]:
import tensorstore as ts
def create_EmptyTensor(filepath, shape, driver = 'zarr3', dtype = 'uint16', fill_value = -np.inf, chunk_shape = [64, 64, 64]):
    out_arr = ts.open({
     'driver': driver,
     'kvstore': 'file://' + str(filepath),
     },
     dtype=dtype,
     fill_value=fill_value,
     chunk_layout=ts.ChunkLayout(chunk_shape=chunk_shape),
     create=True,
     shape=list(shape)).result()
    return out_arr

In [ ]:
# def create_EmptyTensor(filepath, shape, dtype = 'uint16', fill_value = -np.inf, chunk_shape = [64, 64, 64]):
#     out_arr = ts.open({
#      'driver': 'zarr3',
#      'kvstore': 'file://' + str(filepath),
#      },
#      dtype=dtype,
#      fill_value=fill_value,
#      chunk_layout=ts.ChunkLayout(chunk_shape=chunk_shape),
#      create=True,
#      shape=list(shape)).result()
#     return out_arr

In [13]:
## in NG space

# ROI bounding box
# roi_bbox_NG = {
#     "x": (-3590, -1919),
#     "y": (17741, 26159),
#     "z": (25643, 49037)
# }


# roi shape
roi_shape_TS = (
    roi_bbox_NG["z"][1] - roi_bbox_NG["z"][0],
    roi_bbox_NG["y"][1] - roi_bbox_NG["y"][0],
    roi_bbox_NG["x"][1] - roi_bbox_NG["x"][0]
)

# Calculate TS-> NG shift values
TS2NG_shift_values = {
    "z": -roi_bbox_NG["z"][0],
    "y": -roi_bbox_NG["y"][0],
    "x": -roi_bbox_NG["x"][0]
}

# Apply shift to ROI bounding box (in TS space)
roi_bbox_TS = {
    "x": (roi_bbox_NG["x"][0] + TS2NG_shift_values["x"], roi_bbox_NG["x"][1] + TS2NG_shift_values["x"]),
    "y": (roi_bbox_NG["y"][0] + TS2NG_shift_values["y"], roi_bbox_NG["y"][1] + TS2NG_shift_values["y"]),
    "z": (roi_bbox_NG["z"][0] + TS2NG_shift_values["z"], roi_bbox_NG["z"][1] + TS2NG_shift_values["z"])
}

# calculate individual tile -> NG shift
tile2NG_shift_values = []
for bbox in tile_bboxes_NG:
    tile2NG_shift_values.append({
        "z": -bbox["z"][0],
        "y": -bbox["y"][0],
        "x": -bbox["x"][0]
    })

# calculate roi bbox for each tile in NG space
roi_bbox_tile_NG = []
for bbox in tile_bboxes_NG:
    z_start = max(roi_bbox_NG["z"][0], bbox["z"][0])
    z_end = min(roi_bbox_NG["z"][1], bbox["z"][1])
    y_start = max(roi_bbox_NG["y"][0], bbox["y"][0])
    y_end = min(roi_bbox_NG["y"][1], bbox["y"][1])
    x_start = max(roi_bbox_NG["x"][0], bbox["x"][0])
    x_end = min(roi_bbox_NG["x"][1], bbox["x"][1])

    # Check if the bounding boxes overlap
    if z_start < z_end and y_start < y_end and x_start < x_end:
        roi_bbox_tile_NG.append({
            "z": (z_start, z_end),
            "y": (y_start, y_end),
            "x": (x_start, x_end)
        })
    else:
        # Append None for non-overlapping regions
        roi_bbox_tile_NG.append(None)

# shift roi_bbox_tile_NG to TS space, using TS2NG_shift_values
roi_bbox_tile_TS = []
for i, bbox in enumerate(roi_bbox_tile_NG):
    if bbox is None:
        roi_bbox_tile_TS.append(None)
        continue
    roi_bbox_tile_TS.append({
        "z": (bbox["z"][0] + TS2NG_shift_values["z"], bbox["z"][1] + TS2NG_shift_values["z"]),
        "y": (bbox["y"][0] + TS2NG_shift_values["y"], bbox["y"][1] + TS2NG_shift_values["y"]),
        "x": (bbox["x"][0] + TS2NG_shift_values["x"], bbox["x"][1] + TS2NG_shift_values["x"])
    })

# shift roi_bbox_tile_NG to tile space, using tile2NG_shift_values
roi_bbox_tile = []
for i, bbox in enumerate(roi_bbox_tile_NG):
    if bbox is None:
        roi_bbox_tile.append(None)
        continue
    roi_bbox_tile.append({
        "z": (bbox["z"][0] + tile2NG_shift_values[i]["z"], bbox["z"][1] + tile2NG_shift_values[i]["z"]),
        "y": (bbox["y"][0] + tile2NG_shift_values[i]["y"], bbox["y"][1] + tile2NG_shift_values[i]["y"]),
        "x": (bbox["x"][0] + tile2NG_shift_values[i]["x"], bbox["x"][1] + tile2NG_shift_values[i]["x"])
    })

In [14]:
roi_bbox_tile_NG

[{'z': (29031, 33102), 'y': (18719, 20782), 'x': (1688, 1905)}]

In [15]:
# out_arr = create_EmptyTensor('/ACdata/Users/wanqing/data/image/TS_Array_vive_H17_ES2_S7c_20241106_tile1_test5.n5', roi_shape_TS, dtype = 'uint16', driver = 'n5', fill_value=0)
out_arr = create_EmptyTensor(zarr_output_path, roi_shape_TS, dtype = 'uint16', fill_value=0)

In [16]:
out_arr.shape

(4071, 2063, 217)

In [17]:
# Load Zarr data for individual tiles and add to tensor_store
# tile_paths = [
#     "/ACdata/Users/kevin/exaspim_ome_zarr/svc_axonal/H17_PO11_S9_20250221/exaSPIM.zarr/Pos0/0/"
# ]

for i, tile_path in enumerate(tile_paths):
    stack = open_ZarrTensor(tile_path, bytes_limit= 100_000_000)
    bbox_tile = roi_bbox_tile[i]
    bbox_tile_TS = roi_bbox_tile_TS[i]
    if bbox_tile is None:
        continue
    stack = stack[0,0,bbox_tile["z"][0]:bbox_tile["z"][1],
                  bbox_tile["y"][0]:bbox_tile["y"][1],
                  bbox_tile["x"][0]:bbox_tile["x"][1]]
    # stack = stack[0,0,:,:,:]

    # # loop through z
    # for z in range(bbox_tile["z"][0], bbox_tile["z"][1]):
    #     out_arr[z+tile2NG_shift_values[i]["z"],
    #             bbox_tile_TS["y"][0]:bbox_tile_TS["y"][1],
    #             bbox_tile_TS["x"][0]:bbox_tile_TS["x"][1]].write(
    #                 stack[z, 
    #                       bbox_tile["y"][0]:bbox_tile["y"][1],
    #                       bbox_tile["x"][0]:bbox_tile["x"][1]].read().result()).result()
    
    out_arr[bbox_tile_TS["z"][0]:bbox_tile_TS["z"][1],
            bbox_tile_TS["y"][0]:bbox_tile_TS["y"][1],
            bbox_tile_TS["x"][0]:bbox_tile_TS["x"][1]].write(
                stack.read().result()+
                out_arr[bbox_tile_TS["z"][0]:bbox_tile_TS["z"][1], 
                        bbox_tile_TS["y"][0]:bbox_tile_TS["y"][1],
                        bbox_tile_TS["x"][0]:bbox_tile_TS["x"][1]].read().result()).result()
    print(i)

0


In [18]:
out_arr

TensorStore({
  'context': {
    'cache_pool': {},
    'data_copy_concurrency': {},
    'file_io_concurrency': {},
    'file_io_sync': True,
  },
  'driver': 'zarr3',
  'dtype': 'uint16',
  'kvstore': {
    'driver': 'file',
    'path': '/ACdata/Users/wanqing/data/image/H17_PO11_S8_20250226_exaSPIM_Pos0_cutout1/H17_PO11_S8_20250226_exaSPIM_Pos0_cutout1.zarr/',
  },
  'metadata': {
    'chunk_grid': {
      'configuration': {'chunk_shape': [64, 64, 64]},
      'name': 'regular',
    },
    'chunk_key_encoding': {'name': 'default'},
    'codecs': [{'configuration': {'endian': 'little'}, 'name': 'bytes'}],
    'data_type': 'uint16',
    'fill_value': 0,
    'node_type': 'array',
    'shape': [4071, 2063, 217],
    'zarr_format': 3,
  },
  'transform': {
    'input_exclusive_max': [[4071], [2063], [217]],
    'input_inclusive_min': [0, 0, 0],
  },
})

In [19]:
import tifffile as tiff
tiff.imwrite(tiff_output_path, out_arr.read().result())